# Estratégia Magic Formula de Joel Greenblatt

A Magic Formula é uma estratégia de investimento que classifica ações com base em dois critérios principais:

- **Qualidade**: medida pelo Retorno sobre o Capital Investido (ROIC)
- **Valor**: medida pelo Rendimento de Lucros (Earnings Yield)

In [1]:
!pip install pandas numpy requests beautifulsoup4

/usr/bin/bash: warning: setlocale: LC_ALL: cannot change locale (en_US.UTF-8): No such file or directory
  Using cached pandas-2.2.3-cp313-cp313-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (89 kB)
  Using cached numpy-2.2.4-cp313-cp313-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (62 kB)
  Using cached yfinance-0.2.55-py2.py3-none-any.whl.metadata (5.8 kB)
  Using cached pytz-2025.2-py2.py3-none-any.whl.metadata (22 kB)
  Using cached tzdata-2025.2-py2.py3-none-any.whl.metadata (1.4 kB)
  Using cached multitasking-0.0.11-py3-none-any.whl.metadata (5.5 kB)
  Using cached frozendict-2.4.6-py313-none-any.whl.metadata (23 kB)
  Using cached peewee-3.17.9-cp313-cp313-linux_x86_64.whl
Using cached pandas-2.2.3-cp313-cp313-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (12.7 MB)
Using cached numpy-2.2.4-cp313-cp313-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (16.1 MB)
Using cached yfinance-0.2.55-py2.py3-none-any.whl (109 kB)
Using cached frozendict-2.4.6-py313-none-

In [2]:
import numpy as np
import pandas as pd
import requests
from bs4 import BeautifulSoup

In [33]:
def obter_acoes_b3():
    url = "https://www.fundamentus.com.br/resultado.php"
    headers = {
        'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64; rv:136.0) Gecko/20100101 Firefox/136.0'
    }

    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.text, 'html.parser')

    tabela = soup.find('table', {'id': 'resultado'})
    dados = []

    for linha in tabela.find_all('tr')[1:]:
        colunas = linha.find_all('td')
        ticker = colunas[0].text.strip()

        # Extrair métricas fundamentalistas
        ev_ebit = colunas[10].text.strip().replace('.', '').replace(',', '.')
        roic = colunas[16].text.strip().replace('.', '').replace(',', '.')
        liquidez = colunas[18].text.strip().replace('.', '').replace(',', '.')
        divida_pl = colunas[19].text.strip().replace('.', '').replace(',', '.')

        # Converter para float, tratando valores inválidos
        try:
            ev_ebit = float(ev_ebit) if ev_ebit != '-' else np.nan
            roic = float(roic[:-1]) if '%' in roic else float(roic)
            liquidez = float(liquidez)
            divida_pl = float(divida_pl)

            # Calcular Earnings Yield (inverso do EV/EBIT)
            earnings_yield = 1 / ev_ebit if ev_ebit > 0 else np.nan

            dados.append({
                'Ticker': ticker,
                'ROIC': roic,
                'EV/EBIT': ev_ebit,
                'Earnings Yield': earnings_yield,
                'Liquidez Media': liquidez,
                'Divida Bruta/PL': divida_pl
            })
        except (ValueError, ZeroDivisionError):
            continue

    return pd.DataFrame(dados)

In [34]:
acoes_df = obter_acoes_b3()
acoes_df

,Ticker,ROIC,EV/EBIT,Earnings Yield,Liquidez Media,Divida Bruta/PL
0,PORP4,-2.08,0.00,NaN,2.239900e+07,0.00
1,POPR4,19.93,0.00,NaN,5.458030e+08,0.82
2,MNSA3,145.70,0.00,NaN,-9.105000e+06,-6.52
3,CFLU4,32.15,0.00,NaN,6.035100e+07,0.06
4,CSTB4,20.11,0.00,NaN,8.420670e+09,0.14
...,...,...,...,...,...,...
986,UBBR4,0.33,0.00,NaN,1.031720e+10,0.00
987,VSTE3,0.06,25.50,0.039216,1.043280e+09,0.33
988,UBBR11,0.33,0.00,NaN,1.031720e+10,0.00
989,UBBR3,0.33,0.00,NaN,1.031720e+10,0.00


In [35]:
def filtrar_acoes(df):
    """Filtra ações conforme recomendação de Greenblatt"""

    # Remover empresas com baixa liquidez (menor que 1M)
    df_filtrado = df[df['Liquidez Media'] > 1_000_000]

    # Remover empresas com dívida muito alta
    df_filtrado = df_filtrado[(df_filtrado['Divida Bruta/PL'] > 0) & (df_filtrado['Divida Bruta/PL'] < 4)]

    return df_filtrado

In [36]:
acoes_df = filtrar_acoes(acoes_df)

In [37]:
def magic_formula(df):
    # Remover dados inválidos
    df = df[df['Earnings Yield'].notna()].copy()

    # Classificar as ações (ranking)
    df['ROIC_Rank'] = df['ROIC'].rank(ascending=False)
    df['EY_Rank'] = df['Earnings Yield'].rank(ascending=False)

    # Calcular o ranking combinado
    df['Magic_Rank'] = df['ROIC_Rank'] + df['EY_Rank']

    # Ordenar pelo ranking combinado (do menor para o maior)
    df_result = df.sort_values('Magic_Rank')

    return df_result

In [38]:
resultado = magic_formula(acoes_df)

# Selecionar as 15 melhores ações
top_acoes = resultado.head(15)
top_acoes

,Ticker,ROIC,EV/EBIT,Earnings Yield,Liquidez Media,Divida Bruta/PL,ROIC_Rank,EY_Rank,Magic_Rank
453,AHEB3,61.47,2.17,0.460829,2.444130e+08,0.02,7.0,25.0,32.0
455,AHEB5,61.47,2.48,0.403226,2.444130e+08,0.02,7.0,30.0,37.0
459,AHEB6,61.47,2.80,0.357143,2.444130e+08,0.02,7.0,32.0,39.0
367,CSPC4,45.33,1.43,0.699301,2.126670e+09,1.48,23.5,16.5,40.0
371,CSPC3,45.33,1.43,0.699301,2.126670e+09,1.48,23.5,16.5,40.0
433,SYNE3,50.87,2.41,0.414938,1.073780e+09,0.76,16.0,28.0,44.0
435,CPFG4,36.99,0.95,1.052632,3.614430e+09,0.42,37.5,7.0,44.5
437,CPFG3,36.99,0.97,1.030928,3.614430e+09,0.42,37.5,8.0,45.5
396,PALF11,46.56,2.84,0.352113,3.055110e+09,3.00,19.0,33.0,52.0
403,PALF5,46.56,2.87,0.348432,3.055110e+09,3.00,19.0,34.0,53.0


In [39]:
# Salvar resultados em um CSV
top_acoes.to_csv('magic_formula_b3_resultado.csv', index=False)
print("\nResultados salvos em 'magic_formula_b3_resultado.csv'")


Resultados salvos em 'magic_formula_b3_resultado.csv'
